In [115]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from tools.MASweb import get_mas_path
from psipy.model import MASOutput
from psipy.data import sample_data
from psipy.model.variable import Variable
import rom_operator_inference as roi
from scipy.interpolate import interp1d, interp2d
from scipy.interpolate import RegularGridInterpolator
from scipy import stats
import numpy as np 

import time
import astropy.units as u
import pandas as pd
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
from datetime import date

In [165]:
df = pd.read_csv("/home/enrico/Space-Weather-ROM-main/data/ACE_GSE_position.txt", sep="\t")

In [173]:
lon_helio=[]
lat_helio=[]
r_helio=[]

for i in range(0,len(df)):
    obstime=datetime.strftime(datetime.strptime(np.array2string(df.loc[i,"Year"])+np.array2string(df.loc[i,"DOY"]), "%Y%j"),"%Y-%m-%d")
    c=SkyCoord(x=df.loc[i,"GSE_X(km)"]*u.km, y=df.loc[i,"GSE_y(km)"]*u.km, z=df.loc[i,"GSE_z(km)"]*u.km, representation_type='cartesian',obstime=obstime,frame=frames.GeocentricSolarEcliptic)
    h=c.transform_to(frames.HeliographicCarrington(observer='sun'))
    lon_helio.append(h.lon/u.deg)
    lat_helio.append(h.lat/u.deg)
    r_helio.append(h.radius.to(u.AU)/u.AU)

In [174]:
df['HC_lon'] = np.array(lon_helio)
df['HC_lat'] = np.array(lat_helio)
df['HC_radius'] = np.array(r_helio)


In [176]:
df.to_csv("/home/enrico/Space-Weather-ROM-main/data/ACE_GSE_HC_position.txt")

In [172]:
h.radius.to(u.AU)/u.AU

<Quantity 1.00128145>

In [175]:
df

,Year,DOY,Secofday,GSE_X(km),GSE_y(km),GSE_z(km),HC_lon,HC_lat,HC_radius
0,1997,238,0,85516.0,-16700.7,11767.4,171.199693,7.071667,1.010024
1,1997,239,0,264549.4,-2167.4,35162.0,157.981802,7.106861,1.008612
2,1997,240,0,379504.7,13558.7,49749.5,144.764166,7.136719,1.007625
3,1997,241,0,471235.9,24665.1,61150.0,131.548939,7.163408,1.006790
4,1997,242,0,552081.4,32688.7,71026.4,118.335503,7.187526,1.006023
...,...,...,...,...,...,...,...,...,...
9025,2022,132,0,1450341.0,-207559.7,123946.6,215.460642,-2.976252,1.000353
9026,2022,133,0,1447901.3,-202995.9,126953.1,202.234605,-2.864003,1.000590
9027,2022,134,0,1445607.7,-198351.0,129872.6,189.007867,-2.751027,1.000825
9028,2022,135,0,1443650.1,-193526.7,132523.1,175.780428,-2.637432,1.001055
